In [142]:
%cd ~/comp_mut
%pwd

/mnt/storage7/gary/comp_mut


'/mnt/storage7/gary/comp_mut'

In [143]:
import csv
import json
import os.path
from tqdm import tqdm
from collections import defaultdict, Counter
import pandas as pd

In [144]:
# Change these!
# metafile = 'tb_data_18_02_2021.csv'
# jsondir = '/Users/jodyphelan//projects/results/'
# tbdbfile = 'tbdb_altered.csv'
# comp_mutfile = 'compensatory_mutations.csv'

metafile = '../metadata/tb_data_18_02_2021.csv'
jsondir = '/mnt/storage7/jody/tb_ena/tbprofiler/freebayes/results/'
# tbdbfile = 'tbdb_altered.csv'
tbdbfile = "../tbdb/tbdb.csv"
comp_mutfile = 'compensatory_mutations.csv'

In [145]:
def resolve_lineages(data):

    data = dict(data)
    while True:
        improved = False
        for child in sorted(data,reverse= True,key=lambda x : len(x)):
            parent = ".".join(child.split(".")[:-1])
            if any([parent == x for x in data]):
                data[parent] = data[parent] + data[child] 
                del data[child]
                improved = True
                break
        if not improved:
            break
    return data

In [146]:
def json_file(jsondir, s):
    """ 
    Given a sample name as input it returns
    the full path to the json file
    """
    return f"{jsondir}/{s}.results.json"

In [147]:
os.path.isfile(metafile)

True

In [148]:
# Create a dictionary with the metadata
# Sample names as keys
meta = {}
for row in tqdm(csv.DictReader(open(metafile))):
    if os.path.isfile(json_file(jsondir, row['wgs_id'])):
        meta[row['wgs_id']] = row

32735it [00:00, 46277.86it/s]


In [149]:
meta

{'SAMEA2534433': {'wgs_id': 'SAMEA2534433',
  'run_accession': 'ERR552130_ERR552131',
  'sample_accession': 'SAMEA2534433',
  'accession': 'SAMEA2534433',
  'study_accession': 'PRJEB6273',
  'project_name': '',
  'study_name': 'merker2015,coll2018,walker2015,manson2017,bradley2015,holt2018',
  'country_code': 'af',
  'collection_date': '',
  'date_of_collection': '2007',
  'patient_id': 'NA',
  'geographic_source': 'Afghanistan',
  'location': '',
  'country': '',
  'rifampicin': '0',
  'isoniazid': '0',
  'ethambutol': '0',
  'pyrazinamide': '0',
  'streptomycin': '0',
  'ofloxacin': 'NA',
  'moxifloxacin': 'NA',
  'levofloxacin': 'NA',
  'amikacin': 'NA',
  'kanamycin': 'NA',
  'capreomycin': 'NA',
  'ciprofloxacin': 'NA',
  'prothionamide': 'NA',
  'ethionamide': '0',
  'clarithromycin': 'NA',
  'clofazimine': 'NA',
  'bedaquiline': 'NA',
  'cycloserine': 'NA',
  'linezolid': 'NA',
  'para_aminosalicylic_acid': 'NA',
  'rifabutin': 'NA',
  'delamanid': 'NA',
  'host_body_site': '',


In [150]:
# Create a simple list of samples
samples = list(meta.keys())
print(len(samples))

32718


In [151]:
compensatory_mutations = defaultdict(set)
for row in csv.DictReader(open(tbdbfile)):
#     if row['Confers'] != 'compensatory': continue
    if row['Gene'] != 'ahpC': continue
    compensatory_mutations[row['Drug']].add((row['Gene'],row['Mutation']))
    
compensatory_mutations

defaultdict(set,
            {'isoniazid': {('ahpC', 'c.-20C>T'),
              ('ahpC', 'c.-39C>T'),
              ('ahpC', 'c.-44T>A'),
              ('ahpC', 'c.-48G>A'),
              ('ahpC', 'c.-48G>T'),
              ('ahpC', 'c.-49T>G'),
              ('ahpC', 'c.-4A>G'),
              ('ahpC', 'c.-51G>T'),
              ('ahpC', 'c.-52C>T'),
              ('ahpC', 'c.-54C>T'),
              ('ahpC', 'c.-57C>T'),
              ('ahpC', 'c.-5G>A'),
              ('ahpC', 'c.-66G>A'),
              ('ahpC', 'c.-74G>A'),
              ('ahpC', 'c.-81C>T'),
              ('ahpC', 'p.Asp33Asn'),
              ('ahpC', 'p.Asp73His'),
              ('ahpC', 'p.Glu76Lys'),
              ('ahpC', 'p.Leu191Arg'),
              ('ahpC', 'p.Phe10Ile'),
              ('ahpC', 'p.Pro2Ser'),
              ('ahpC', 'p.Thr5Ile')}})

In [152]:
samples = samples[:1000]

In [170]:
# Load mutation data using ('gene','change') as keys
# genes = set(target_df['gene'])
genes = ('ahpC', 'katG', 'fabG1')
mutation2sample = defaultdict(set)
sample2mutation = defaultdict(set)
resistance_mutations = defaultdict(set)
for s in tqdm(samples):
    data = json.load(open(json_file(jsondir, s)))
    
    if ';' in data['sublin']: continue
        
    if 'sublin' not in data.keys():
        print(samp)
    
    meta[s]['drtype'] = data['drtype']
    meta[s]['sublin'] = data['sublin']
    
    for var in data['dr_variants'] + data['other_variants']:
        if var['gene'] not in genes: continue
        if var['freq'] < 0.7: continue
        if var['type']=='synonymous_variant': continue
        key = (var['gene'],var['change'])
        mutation2sample[key].add(s)
        sample2mutation[s].add(key)
        if "drugs" in var:
            for d in var["drugs"]:
                if key in compensatory_mutations[d["drug"]]: continue
                resistance_mutations[d["drug"]].add(key)

100%|██████████| 1000/1000 [00:00<00:00, 3672.85it/s]


In [173]:
x = json.load(open(json_file(jsondir, 'SRR6339667')))

x['sublin']

'lineage3.1.2;La1.1'

In [161]:
for i, samp in enumerate(samples):
    if 'sublin' not in meta[samp].keys():
        print(i)
        print(samp)

meta['SRR6339667']

258
SRR6339667
267
SRR6339644
269
SRR6339642
270
SRR6339641
281
SRR5817467
412
ERR1213946
420
ERR2516172
439
ERR2516191
458
ERR2516210
463
ERR2516215
518
ERR2516271
553
ERR2516306
554
ERR2516307
557
ERR2516311
570
ERR2516324
620
ERR2516376
635
ERR2516392
648
ERR1034658
657
ERR1034667
995
SRR6153204


{'wgs_id': 'SRR6339667',
 'run_accession': 'SRR6339667',
 'sample_accession': 'SAMN07978244',
 'accession': 'SAMN07978244',
 'study_accession': 'NA',
 'project_name': '',
 'study_name': 'walker2018',
 'country_code': 'au',
 'collection_date': '01/01/2013',
 'date_of_collection': 'NA',
 'patient_id': 'NA',
 'geographic_source': 'Australia',
 'location': '33.868 S 151.209 E',
 'country': 'Australia',
 'rifampicin': '0',
 'isoniazid': '1',
 'ethambutol': '1',
 'pyrazinamide': '0',
 'streptomycin': 'NA',
 'ofloxacin': 'NA',
 'moxifloxacin': 'NA',
 'levofloxacin': 'NA',
 'amikacin': 'NA',
 'kanamycin': 'NA',
 'capreomycin': 'NA',
 'ciprofloxacin': 'NA',
 'prothionamide': 'NA',
 'ethionamide': 'NA',
 'clarithromycin': 'NA',
 'clofazimine': 'NA',
 'bedaquiline': 'NA',
 'cycloserine': 'NA',
 'linezolid': 'NA',
 'para_aminosalicylic_acid': 'NA',
 'rifabutin': 'NA',
 'delamanid': 'NA',
 'host_body_site': '',
 'environment_material': '',
 'sample_alias': '13-1149-0008',
 'sample_title': 'This sam

In [137]:
mutation2sample

defaultdict(set,
            {('katG', 'p.Arg463Leu'): {'ERR1034666',
              'ERR1213947',
              'ERR1213948',
              'ERR1213949',
              'ERR2432985',
              'ERR2433002',
              'ERR2433005',
              'ERR2433012',
              'ERR2516177',
              'ERR2516180',
              'ERR2516182',
              'ERR2516183',
              'ERR2516186',
              'ERR2516188',
              'ERR2516189',
              'ERR2516194',
              'ERR2516196',
              'ERR2516198',
              'ERR2516203',
              'ERR2516207',
              'ERR2516209',
              'ERR2516213',
              'ERR2516214',
              'ERR2516217',
              'ERR2516219',
              'ERR2516226',
              'ERR2516229',
              'ERR2516230',
              'ERR2516239',
              'ERR2516240',
              'ERR2516243',
              'ERR2516245',
              'ERR2516249',
              'ERR2516252',
      

In [138]:
sample2mutation

defaultdict(set,
            {'SAMEA2534433': {('katG', 'p.Arg463Leu')},
             'ERR1465765': {('katG', 'p.Ser315Thr')},
             'SAMEA3715554': {('katG', 'p.Ser315Thr')},
             'ERR1465906': {('fabG1', 'c.-278T>C')},
             'ERR1193661': {('katG', 'p.Ser315Thr')},
             'ERR1193674': {('katG', 'p.Ser315Thr')},
             'ERR757145': {('katG', 'p.Ser315Thr')},
             'ERR757146': {('katG', 'p.Ser315Thr')},
             'ERR757147': {('katG', 'p.Ser315Thr')},
             'ERR757148': {('katG', 'p.Ser315Thr')},
             'SAMEA3231528': {('katG', 'p.Ser315Thr')},
             'ERR757150': {('katG', 'p.Ser315Thr')},
             'ERR757151': {('katG', 'p.Ser315Thr')},
             'ERR757159': {('katG', 'p.Ser315Thr')},
             'ERR757160': {('katG', 'p.Ser315Thr')},
             'ERR757161': {('katG', 'p.Ser315Thr')},
             'ERR757152': {('katG', 'p.Ser315Thr')},
             'ERR757162': {('katG', 'p.Ser315Thr')},
             'ERR

In [139]:
resistance_mutations

defaultdict(set,
            {'isoniazid': {('fabG1', 'c.-15C>T'),
              ('fabG1', 'c.-17G>T'),
              ('fabG1', 'c.-8T>A'),
              ('fabG1', 'c.-8T>C'),
              ('katG', 'c.-104_*2462del'),
              ('katG', 'c.1328dupC'),
              ('katG', 'c.1338_1342dupCCACG'),
              ('katG', 'c.316dupG'),
              ('katG', 'c.54delC'),
              ('katG', 'c.6_7delCG'),
              ('katG', 'p.Ala106Val'),
              ('katG', 'p.Ala110Val'),
              ('katG', 'p.Ala424Val'),
              ('katG', 'p.Arg104Gln'),
              ('katG', 'p.Asn138His'),
              ('katG', 'p.Met257Ile'),
              ('katG', 'p.Ser302Arg'),
              ('katG', 'p.Ser315Arg'),
              ('katG', 'p.Ser315Asn'),
              ('katG', 'p.Ser315Gly'),
              ('katG', 'p.Ser315Thr'),
              ('katG', 'p.Thr275Ala'),
              ('katG', 'p.Thr85Pro'),
              ('katG', 'p.Trp191Arg'),
              ('katG', 'p.Trp90*'),
    

In [154]:
drug = 'isoniazid'
potential_resistance_mutations = set()
for s in tqdm(samples):
    comp_var = [var for var in sample2mutation[s] if var in compensatory_mutations[drug]]
    res_var = [var for var in sample2mutation[s] if var in resistance_mutations[drug]]
    other_vars = [var for var in sample2mutation[s] if var not in compensatory_mutations[drug] and \
                  var not in resistance_mutations[drug]]
    # If there is at least one comp variants and there are no resistance variants
    if len(comp_var)>0 and len(res_var)==0:
        # If there are no 'other' variants print the sample and the comp variants
        if len(other_vars)==0:
            print("Sample, comp. variant")
            print(s,comp_var)
        # Store the 'other' vars as potential resistance variants
        for var in other_vars:
            potential_resistance_mutations.add(var)


100%|██████████| 1000/1000 [00:00<00:00, 361515.60it/s]

Sample, comp. variant
ERR2516197 [('ahpC', 'p.Leu191Arg')]
Sample, comp. variant
ERR2516199 [('ahpC', 'c.-52C>T')]


In [155]:
potential_resistance_mutations

{('katG', 'p.Asp735Tyr'),
 ('katG', 'p.Gln525Leu'),
 ('katG', 'p.Gly299Ser'),
 ('katG', 'p.Thr394Pro')}

In [156]:
def get_counts(meta,samples,col):
    return dict(Counter([meta[s][col] for s in samples]))

def get_meta_proportion(meta,samples,column,targets):
    tmp = get_counts(meta,samples,column)
    target_count = sum([tmp.get(c,0) for c in targets])
    return target_count/sum(tmp.values())
    

In [159]:
get_counts(meta, samples, 'sublin')

KeyError: 'sublin'

In [246]:
tmp = set()
for var in potential_resistance_mutations:
    samps = mutation2sample[var]

    if len(samps)<3: continue
    tmp.add(var)
    resistant_dst_proportion = get_meta_proportion(meta,samps,drug,['1'])
    sensitive_geno_proportion = get_meta_proportion(meta,samps,'drtype',['Sensitive'])
    num_lins = len(set(get_counts(meta,samps,'sublin')))
    print(var,len(samps),resistant_dst_proportion,sensitive_geno_proportion,num_lins)
    print(Counter(resolve_lineages(get_counts(meta,samps,'sublin'))))
    print()
    

('katG', 'p.Ala312Glu') 4 0.5 0.0 1
Counter({'lineage2.2.1': 4})

('fabG1', 'c.-102G>A') 254 0.027559055118110236 0.6614173228346457 10
Counter({'lineage5': 253, 'lineage1.1.2': 1})

('ahpC', 'p.Leu191Pro') 3 1.0 0.0 2
Counter({'lineage2.2.1': 3})

('katG', 'p.Arg484His') 5 1.0 0.2 3
Counter({'lineage3': 2, 'lineage4.5': 2, 'lineage4.3.3': 1})

('katG', 'p.Arg78Pro') 4 0.25 0.25 2
Counter({'lineage2.2.1': 2, 'lineage4.8': 2})

('katG', 'p.Asp142Gly') 9 0.8888888888888888 0.3333333333333333 4
Counter({'lineage2.2.1': 6, 'lineage1.2.2.2': 1, 'lineage4.3.2.1': 1, 'lineage3': 1})

('katG', 'p.Arg385Pro') 3 0.6666666666666666 0.0 1
Counter({'lineage4.5': 3})

('katG', 'c.984_998del') 3 0.6666666666666666 0.0 1
Counter({'lineage4.5': 3})

('katG', 'p.Pro232Ser') 7 0.7142857142857143 0.0 4
Counter({'lineage2.2.1': 3, 'lineage2.2.2': 2, 'lineage4.1.2.1': 1, 'lineage5.1': 1})

('katG', 'p.Asp189Gly') 6 0.3333333333333333 0.16666666666666666 4
Counter({'lineage2.2.1': 4, 'lineage4.1.2.1': 1, 'li

{'ERR2503557',
 'ERR718194',
 'ERR718225',
 'ERR718250',
 'ERR718297',
 'ERR718305',
 'ERR718377',
 'ERR718378',
 'ERR718440',
 'ERR718469',
 'ERR718483',
 'ERR718492',
 'ERR718503',
 'ERR718530',
 'ERR751635',
 'ERR751646',
 'ERR751701',
 'ERR751727',
 'ERR751856',
 'ERR751868',
 'ERR751920',
 'ERR751943',
 'ERR752010',
 'ERR752016',
 'ERR752070',
 'ERR752089',
 'ERR752110',
 'ERR752193',
 'ERR752246',
 'ERR752254',
 'ERR767982',
 'ERR768042',
 'ERR768048',
 'SAMN02599150',
 'SRR1723442',
 'SRR1723443',
 'SRR8237404'}